In [1]:
import pandas as pd
import numpy as np
import datetime

In [53]:
data = pd.read_csv('first_df_with_RM.csv', parse_dates=["time"])

In [54]:
data

,time,ticker,open,high,low,close,volume,time.1,rv,gamma,RM
0,2019-09-30 07:00:00,ATVI,52.117346,52.117346,51.988808,51.988808,698.0,2019-09-30 07:00:00,NaN,NaN,0.000000
1,2019-09-30 07:30:00,ATVI,52.058021,52.058021,52.058021,52.058021,199.0,2019-09-30 07:30:00,0.004790,NaN,0.000000
2,2019-09-30 08:00:00,ATVI,52.058021,52.058021,52.058021,52.058021,199.0,2019-09-30 08:00:00,0.000000,0.000000,NaN
3,2019-09-30 08:30:00,ATVI,51.899820,51.899820,51.899820,51.899820,132.0,2019-09-30 08:30:00,0.025028,-0.000000,0.000000
4,2019-09-30 09:00:00,ATVI,52.107360,52.107360,52.107360,52.107360,150.0,2019-09-30 09:00:00,0.043073,-0.032833,-0.032833
...,...,...,...,...,...,...,...,...,...,...,...
20122,2021-09-17 16:00:00,ATVI,79.355000,79.830000,79.295000,79.560000,1853014.0,2021-09-17 16:00:00,0.040000,0.062000,0.062000
20123,2021-09-17 16:30:00,ATVI,79.560000,79.560000,79.560000,79.560000,692564.0,2021-09-17 16:30:00,0.000000,0.000000,0.000000
20124,2021-09-17 17:00:00,ATVI,79.000000,79.020000,79.000000,79.010000,1236.0,2021-09-17 17:00:00,0.302500,-0.000000,0.000000
20125,2021-09-17 17:30:00,ATVI,79.200000,79.200000,79.200000,79.200000,201.0,2021-09-17 17:30:00,0.036100,-0.104500,-0.104500


In [55]:
from scipy.optimize import minimize

In [56]:
def log_Q2(alphaR, betaR, rv, RM):
    T = len(rv)
    mu_t = T**0.5 * np.sum/(RM[:int(T**0.5)])
    ll -= 0.5*(np.log(mu_t) + RM[0]/mu_t)
    
    mu = rv.sum().mean()
    muR = RM.sum().mean()
    k = mu / muR
    for i in range(1, T):
        mu_t = muR*(1 - alphaR - betaR) + alphaR*RM[i] + betaR*mu_t
        ll -= 0.5*(np.log(mu_t) + RM[i]/mu_t)
        
    return ll

In [57]:
def log_Q1(alpha, beta, rv, RM):
    T = len(rv)
    h = T**0.5 * np.sum(rv[:int(T**0.5)])
    ll = 0
    h_list = [h]
    
    mu = rv.sum().mean()
    muR = RM.sum().mean()
    k = mu / muR
#     print('k', k)
    for i in range(1, T):
        h = mu*(1 - alpha*k - beta) + alpha*RM[i-1] + beta*h
        h_list.append(h)
        ll += 0.5*(np.log(h) + rv[i]/h)
        
    print(np.round(h_list, 3))
    return ll

In [60]:
df = data[data['time'].dt.date == datetime.datetime(2019,9, 30).date()]
rv = df['rv'].fillna(0)
RM = df['RM'].fillna(0)

In [61]:
fun = lambda params: log_Q1(params[0], params[1], rv, RM)
bnds = ((0, None), (0, 1))
res = minimize(fun, (0.3, 0.5), method='L-BFGS-B', bounds=bnds, tol=1e-6)

[0.143 0.816 1.152 1.321 1.405 1.437 1.462 1.476 1.482 1.486 1.489 1.492
 1.487 1.49  1.487 1.494 1.487 1.488 1.488 1.489 1.493 1.484 1.484]
[0.143 0.816 1.152 1.321 1.405 1.437 1.462 1.476 1.482 1.486 1.489 1.492
 1.487 1.49  1.487 1.494 1.487 1.488 1.488 1.489 1.493 1.484 1.484]
[0.143 0.816 1.152 1.321 1.405 1.437 1.462 1.476 1.482 1.486 1.489 1.492
 1.487 1.49  1.487 1.494 1.487 1.488 1.488 1.489 1.493 1.484 1.484]
[0.143 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391
 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391]
[0.143 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391
 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391]
[0.143 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391
 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391]
[0.143 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391
 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391 0.391]
[0.143 0.391 

In [64]:
res

      fun: -20.02531584352543
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([1349.99064372, -183.54937495])
  message: 'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 30
      nit: 2
     njev: 10
   status: 0
  success: True
        x: array([0., 1.])

In [62]:
fun((1000000, 0.4))

[1.43000000e-01 1.82926283e+06 2.56096790e+06 2.85364993e+06
 2.97072275e+06 2.98471884e+06 3.01906670e+06 3.03834858e+06
 3.04423559e+06 3.04717698e+06 3.05297288e+06 3.05959285e+06
 3.04229700e+06 3.05317169e+06 3.04327248e+06 3.06529991e+06
 3.03985919e+06 3.04520645e+06 3.04734535e+06 3.05015619e+06
 3.06144797e+06 3.02959651e+06 3.03249815e+06]


163.82399285597327

In [63]:
fun(res.x)

[0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143
 0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143 0.143]


-20.02531584352543